In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pylab as plt
%matplotlib inline

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.datasets import make_classification
# X为样本特征，y为样本类别输出， 共10000个样本，每个样本20个特征，输出有2个类别，没有冗余特征，每个类别一个簇
X, y = make_classification(n_samples=10000, n_features=20, n_redundant=0,
                             n_clusters_per_class=1, n_classes=2, flip_y=0.1)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

(7500, 20)
(7500,)
(2500, 20)
(2500,)


## XGBoost 使用sklearn风格的参数

In [4]:
xgbClf = xgb.XGBClassifier(max_depth=5,learning_rate= 0.5, verbosity=1, objective='binary:logistic',random_state=1, use_label_encoder=False)

In [5]:
xgbClf.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="error", eval_set=[(X_test, y_test)])

[0]	validation_0-error:0.11000
[1]	validation_0-error:0.11280
[2]	validation_0-error:0.11120
[3]	validation_0-error:0.11400
[4]	validation_0-error:0.11120
[5]	validation_0-error:0.11160
[6]	validation_0-error:0.10960
[7]	validation_0-error:0.11280
[8]	validation_0-error:0.11160
[9]	validation_0-error:0.11000
[10]	validation_0-error:0.11200
[11]	validation_0-error:0.11240
[12]	validation_0-error:0.11320
[13]	validation_0-error:0.11160
[14]	validation_0-error:0.11320
[15]	validation_0-error:0.11320
[16]	validation_0-error:0.11200


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.5, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', use_label_encoder=False,
              validate_parameters=1, verbosity=1)

<h1>使用sklearn网格搜索调参</h1>

一般固定步长，先调好框架参数n_estimators，再调弱学习器参数max_depth，min_child_weight,gamma等，接着调正则化相关参数subsample，colsample_byXXX, reg_alpha以及reg_lambda,最后固定前面调好的参数，来调步长learning_rate

In [6]:
gsCv = GridSearchCV(xgbClf, {'max_depth': [4,5,6], 'n_estimators': [5,10,20]})
gsCv.fit(X_train,y_train)

[10:41:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:41:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:41:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:41:42] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

[10:41:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:41:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:41:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[10:41:44] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective

GridSearchCV(estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0, gpu_id=-1,
                                     importance_type='gain',
                                     interaction_constraints='',
                                     learning_rate=0.5, max_delta_step=0,
                                     max_depth=5, min_child_weight=1,
                                     missing=nan, monotone_constraints='()',
                                     n_estimators=100, n_jobs=8,
                                     num_parallel_tree=1, random_state=1,
                                     reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, subsample=1,
                                     tree_method='exact',
                                     use_label_encoder=False,
                                  

In [7]:
print(gsCv.best_score_)
print(gsCv.best_params_)

0.8968
{'max_depth': 4, 'n_estimators': 5}


In [9]:
xgbClf2 = xgb.XGBClassifier(max_depth=4, n_estimators=5, verbosity=1, objective='binary:logistic', random_state=1)
gsCv2 = GridSearchCV(xgbClf2, {'learning_rate ': [0.3,0.5,0.7]})
gsCv2.fit(X_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Fr

XGBoostError: [10:42:22] /Users/travis/build/dmlc/xgboost/src/learner.cc:550: Invalid parameter "learning_rate " contains whitespace.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x00000001293ef074 dmlc::LogMessageFatal::~LogMessageFatal() + 116
  [bt] (1) 2   libxgboost.dylib                    0x00000001294ad1f4 xgboost::LearnerConfiguration::ValidateParameters() + 2340
  [bt] (2) 3   libxgboost.dylib                    0x00000001294a0869 xgboost::LearnerConfiguration::Configure() + 1561
  [bt] (3) 4   libxgboost.dylib                    0x00000001294a0c68 xgboost::LearnerImpl::UpdateOneIter(int, std::__1::shared_ptr<xgboost::DMatrix>) + 120
  [bt] (4) 5   libxgboost.dylib                    0x00000001293e5ce9 XGBoosterUpdateOneIter + 137
  [bt] (5) 6   libffi.dylib                        0x00007fff6c189905 ffi_call_unix64 + 85
  [bt] (6) 7   ???                                 0x00007ffee25f8ec0 0x0 + 140732696334016



In [10]:
print(gsCv2.best_score_)
print(gsCv2.best_params_)

nan
{'learning_rate ': 0.3}


In [11]:
xgbClf3 = xgb.XGBClassifier(max_depth=4,learning_rate= 0.3, verbosity=1, objective='binary:logistic',n_estimators=5)
xgbClf3.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="error", eval_set=[(X_test, y_test)])

[0]	validation_0-error:0.11360
[1]	validation_0-error:0.11040
[2]	validation_0-error:0.10600
[3]	validation_0-error:0.10640
[4]	validation_0-error:0.10480


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.3, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=5, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=1)

In [12]:
from sklearn.metrics import accuracy_score
pred_test_new = xgbClf3.predict(X_test)
print (accuracy_score(y_test, pred_test_new))

0.8952
